In [1]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

In [2]:
%%writefile fairy_interaction_agent.py
from agents.fairy.util import measure_latency
import time
from agents.fairy.fairy_state import (
    FairyInteractionState,
    FairyInterationIntentType,
    FairyInterationIntentOutput,
)
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
from enums.LLM import LLM
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from typing import List
from core.common import get_inventory_items
from agents.fairy.util import get_groq_llm_lc
from agents.fairy.fairy_state import FairyItemUseOutput
from agents.fairy.interaction.fairy_interaction_model_logics import ItemEmbeddingLogic, IsItemUseEmbeddingLogic, FairyInteractionIntentModel

item_embedding_logic = ItemEmbeddingLogic()
is_item_use_embedding_logic = IsItemUseEmbeddingLogic()
fairy_interaction_intent_model = FairyInteractionIntentModel()

llm = get_groq_llm_lc(max_token=2)
def _clarify_intent(query):
    # interation_intent_prompt = PromptManager(
    #     FairyPromptType.FAIRY_INTERACTION_INTENT
    # ).get_prompt(question=query)
    # parser_llm = llm.with_structured_output(FairyInterationIntentOutput)
    # intent_output: FairyInterationIntentOutput = parser_llm.invoke(
    #     interation_intent_prompt
    # )
    
    raw_labels, _ = fairy_interaction_intent_model.predict(query)
    enum_list = FairyInteractionIntentModel.parse_intents_to_enum(raw_labels)    
    intent_output = FairyInterationIntentOutput(intents=enum_list)
    return intent_output


def analyze_intent(state: FairyInteractionState):
    start = time.perf_counter()

    last = state["messages"][-1]
    last_message = last.content
    intent_output: FairyInterationIntentOutput = _clarify_intent(last_message)

    latency = time.perf_counter() - start
    return {
        "intent_types": intent_output.intents,
        "latency_analyze_intent": latency,
    }


# LLM Call을 한번이라도 줄이기 위해 의도 분석과 함께 병렬 호출 Node
def create_temp_use_item_id(state: FairyInteractionState):
    start = time.perf_counter()
    last = state["messages"][-1]
    last_message = last.content
    inventory = state["inventory"]
    my_items = get_inventory_items(inventory)
    item_use_prompt = PromptManager(FairyPromptType.FAIRY_ITEM_USE).get_prompt(
        inventory_items=my_items, question=last_message
    )

    # parser_llm = llm.with_structured_output(FairyItemUseOutput)
    output = FairyItemUseOutput(item_id=int(llm.invoke(item_use_prompt).content))  # for type hinting
    latency = time.perf_counter() - start
    return {"temp_use_item_id": output.item_id,
            "latency_create_temp_use_item_id": latency}
    # item_id = item_embedding_logic.pick_items(last_message, inventory, top_k=1)[0]
    # latency = time.perf_counter() - start
    # return {
    #     "temp_use_item_id": item_id,
    #     "latency_create_temp_use_item_id": latency,
    # }

def check_use_item(state:FairyInteractionState): 
    start = time.perf_counter()
    last = state["messages"][-1]
    last_message = last.content
    is_item_use = is_item_use_embedding_logic.is_item_use(last_message)
    latency = time.perf_counter() - start
    return {
        "is_item_use": is_item_use,
        "latency_check_use_item": latency,
    }


def create_interation(state: FairyInteractionState):
    intent_types: List[FairyInterationIntentType] = state["intent_types"]
    is_item_use = state["is_item_use"]
    
    item_id = None
    if FairyInterationIntentType.INVENTORY_ITEM_USE in intent_types and (is_item_use != False):
        item_id = state["temp_use_item_id"]

    room_light = 0
    if FairyInterationIntentType.LIGHT_ON_ROOM in intent_types:
        room_light = 1

    if FairyInterationIntentType.LIGHT_OFF_ROOM in intent_types:
        room_light = 2

    isCheckNextRoom = FairyInterationIntentType.MOVE_NEXT_ROOM in intent_types

    return {
        "useItemId": item_id,
        "roomLight": room_light,
        "isCheckNextRoom": isCheckNextRoom,
    }


graph_builder = StateGraph(FairyInteractionState)
graph_builder.add_node("analyze_intent", analyze_intent)
graph_builder.add_node("create_temp_use_item_id", create_temp_use_item_id)
graph_builder.add_node("create_interation", create_interation)
graph_builder.add_node("check_use_item", check_use_item)


graph_builder.add_edge(START, "analyze_intent")
graph_builder.add_edge(START, "create_temp_use_item_id")
graph_builder.add_edge(START, "check_use_item")

graph_builder.add_edge("analyze_intent", "create_interation")
graph_builder.add_edge("create_temp_use_item_id", "create_interation")
graph_builder.add_edge("check_use_item", "create_interation")

graph_builder.add_edge("create_interation", END)
graph = graph_builder.compile()

Overwriting fairy_interaction_agent.py


In [3]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))
from langchain_core.messages import HumanMessage
from agents.fairy.interaction.fairy_interaction_agent import graph



/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 285975.27it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [4]:
result = graph.invoke(
    {
        "inventory": [0, 21, 42],
        "messages": [HumanMessage("내 인벤토리중에 가장 좋은 무기 써줘")],
    }
)
result


{'messages': [HumanMessage(content='내 인벤토리중에 가장 좋은 무기 써줘', additional_kwargs={}, response_metadata={}, id='7c481997-90e6-4d39-a1a7-e144b9873b6c')],
 'inventory': [0, 21, 42],
 'roomLight': 0,
 'isCheckNextRoom': False,
 'useItemId': None,
 'is_item_use': False,
 'intent_types': [<FairyInterationIntentType.INVENTORY_ITEM_USE: 'INVENTORY_ITEM_USE'>],
 'temp_use_item_id': 42,
 'latency_analyze_intent': 0.1029696250006964,
 'latency_create_temp_use_item_id': 0.30640891599978204,
 'latency_check_use_item': 0.1706945830010227}

In [ ]:
print("방불 키기 여부",result['roomLight'])
print("다음방 이동 여부",result['isCheckNextRoom'])
print("아이템 사용",result['useItemId'])

In [ ]:
result

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))
result = graph.invoke(
    {
        "inventory": [66,64,65],
        "messages": [HumanMessage("뿅망치좀 골라줘")],
    }
)

result

In [ ]:
print("방불 키기 여부",result['roomLight'])
print("다음방 이동 여부",result['isCheckNextRoom'])
print("아이템 사용",result['useItemId'])